In [3]:
import pandas as pd
import numpy as np
file_path = 'cleaned_samsung_phones.csv'
df_samsung = pd.read_csv(file_path)

# Display the first few rows to understand its structure
df_samsung.head()

,model_name,popularity,best_price,highest_price,screen_size,memory_size,battery_size,release_date
0,G920F Galaxy S6 32GB (White Pearl),845,3529.0,3959.0,5.1,32.0,2550.0,3650
1,Galaxy A01 2/16GB Black (SM-A015FZKD),1163,2940.0,3158.0,5.7,16.0,3000.0,1883
2,Galaxy A01 Core 1/16GB Black (SM-A013FZKD),1073,2344.0,2553.0,5.3,16.0,3000.0,1670
3,Galaxy A02s 3/32GB Blue (SM-A025FZBE),1078,3776.0,3799.0,6.5,32.0,5000.0,1517
4,Galaxy A10s 2019 SM-A107F 2/32GB Black (SM-A10...,1129,3348.0,3999.0,6.2,32.0,4000.0,2036


In [ ]:
# Updated normalization using x_ij / sum_j(x_ij)
def normalize_matrix(df):
    normalized_matrix = df.copy()
    for col in df.columns:
        if col != 'model_name' and df[col].dtype in ['float64', 'int64']:
            # Normalize using x_ij / sum_j(x_ij)
            normalized_matrix[col] = df[col] / df[col].sum()
    return normalized_matrix

# Reapply the normalization
normalized_df_updated = normalize_matrix(df_samsung.drop(columns=['model_name']))

# Step 1: Entropy method to calculate weights (unchanged)
def calculate_entropy_weights(normalized_df):
    # Calculate p_ij * ln(p_ij)
    entropy_matrix = normalized_df * np.log(normalized_df.replace(0, np.nan))
    entropy_matrix = entropy_matrix.fillna(0)
    
    # Calculate entropy for each criterion
    m = len(normalized_df)
    k = 1 / np.log(m)
    entropy_values = -k * entropy_matrix.sum(axis=0)
    
    # Calculate weights
    weights_entropy = 1 - entropy_values
    weight_sum = weights_entropy.sum()
    weights_entropy = weights_entropy / weight_sum
    return weights_entropy

# Step 2: AHP method to calculate weights and Consistency Ratio (CR)
def calculate_ahp_weights_and_cr(normalized_df):
    # Calculate the weight using mean of the normalized matrix
    weights_ahp = normalized_df.mean(axis=0)
    # weight_sum = weights_ahp.sum()
    # weights_ahp = weights_ahp / weight_sum
    
    # Now calculate the Consistency Ratio (CR)
    n = len(weights_ahp)
    
    # Create the comparison matrix (simplified for illustration)
    comparison_matrix = np.ones((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            comparison_matrix[i, j] = 1 / comparison_matrix[j, i]
    
    # Calculate the lambda_max value
    weighted_sum = np.dot(normalized_df, weights_ahp)
    lambdas = weighted_sum / weights_ahp
    lambda_max = np.max(lambdas)
    
    # Calculate Consistency Index (CI)
    CI = (lambda_max - n) / (n - 1)
    
    # Calculate Random Index (RI) for n = number of criteria
    RI = {1: 0, 2: 0, 3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}.get(n, 1.5)
    
    # Calculate Consistency Ratio (CR)
    CR = CI / RI
    
    return weights_ahp, CR

# Calculate weights and CR for both Entropy and AHP methods
weights_entropy_updated = calculate_entropy_weights(normalized_df_updated)
weights_ahp_updated, CR_ahp_updated = calculate_ahp_weights_and_cr(normalized_df_updated)

# Display results
weights_entropy_updated, weights_ahp_updated, CR_ahp_updated

ValueError: operands could not be broadcast together with shapes (142,) (7,) 